In [ ]:
catalog.list()

In [26]:
df = catalog.load('processed_p18_19_5')


2023-05-23 21:13:10,776 - kedro.io.data_catalog - INFO - Loading data from 'processed_p18_19_5' (CSVDataSet)...


In [27]:
df.columns

Index(['other_entities', 'headcount_female', 'headcount_male',
       'headcount_total', 'percentage_female', 'percentage_male'],
      dtype='object')

In [ ]:
validator = df

In [ ]:
df.head()

In [1]:
# Import GX and instantiate a Data Context
import great_expectations as gx

context = gx.get_context()


2023-05-23 21:04:25,912 - py.warnings - WARNING - C:\Users\gdbt0\anaconda3\envs\Data-preparation-pipeline\lib\site-packages\great_expectations\compatibility\sqlalchemy.py:20: UserWarning: SQLAlchemy v2.0.0 or later is not yet supported by Great Expectations.
  warnings.warn(

2023-05-23 21:04:26,346 - py.warnings - WARNING - C:\Users\gdbt0\anaconda3\envs\Data-preparation-pipeline\lib\site-packages\great_expectations\compatibility\sqlalchemy.py:20: UserWarning: SQLAlchemy v2.0.0 or later is not yet supported by Great Expectations.
  warnings.warn(

2023-05-23 21:04:27,862 - great_expectations.data_context.data_context.file_data_context - INFO - FileDataContext loading fluent config
2023-05-23 21:04:27,865 - great_expectations.datasource.fluent.config - INFO - Loading 'datasources' ->
[{'assets': [...],
  'boto3_options': {},
  'bucket': 'processed-data-7374046',
  'name': 'processed_datasource',
  'type': 'pandas_s3'}]
2023-05-23 21:04:27,900 - great_expectations.data_context.data_conte

In [2]:
# Retrieve the Data Asset
datasource_name = "processed_datasource"
asset = context.get_datasource(datasource_name).get_asset("processed_data_asset")

In [3]:
# Build a Batch Request
batch_request = asset.build_batch_request()

In [5]:
# Create an ExpectationSuite

suite = context.add_expectation_suite(expectation_suite_name="processed_data_suite")

In [18]:
# Create Expectation Configurations

from great_expectations.core.expectation_configuration import ExpectationConfiguration

expectation_configuration_1 = ExpectationConfiguration(
    expectation_type="expect_column_values_to_not_be_null",
    kwargs={
        "column": "other_entities",
    },
)
suite.add_expectation(expectation_configuration=expectation_configuration_1)

{"kwargs": {"column": "other_entities"}, "meta": {}, "expectation_type": "expect_column_values_to_not_be_null"}

In [19]:
expectation_suite_name = "processed_data_suite"
# Save your Expectations for future use
context.save_expectation_suite(expectation_suite=suite)

2023-05-23 21:11:33,759 - py.warnings - WARNING - C:\Users\gdbt0\anaconda3\envs\Data-preparation-pipeline\lib\site-packages\great_expectations\data_context\data_context\abstract_data_context.py:435: DeprecationWarning: save_expectation_suite is deprecated as of v0.15.48 and will be removed in v0.18. Please use update_expectation_suite or add_or_update_expectation_suite instead.
  warnings.warn(



'C:\\Users\\gdbt0\\Projects\\Data-preparation-pipeline\\data-processing-pipeline\\great_expectations\\expectations/processed_data_suite.json'

In [20]:
# Instantiate a Validator
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name=expectation_suite_name
)
validator.head()

2023-05-23 21:11:34,361 - great_expectations.datasource.fluent.fluent_base_model - INFO - PandasS3Datasource.dict() - substituting config values
2023-05-23 21:11:34,371 - great_expectations.datasource.data_connector.batch_filter - INFO - batch_slice: None was parsed to: slice(0, None, None)
2023-05-23 21:11:34,374 - great_expectations.datasource.data_connector.batch_filter - INFO - batch_slice: slice(0, None, None) was parsed to: slice(0, None, None)
2023-05-23 21:11:34,375 - great_expectations.datasource.fluent.fluent_base_model - INFO - CSVAsset.dict() - substituting config values


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,other_entities,headcount_female,headcount_male,headcount_total,percentage_female,percentage_male
0,Legal Aid Queensland,480,143,623,0.770465,0.229535
1,Office of the Health Ombudsman,100,42,142,0.704225,0.295775
2,Queensland Art Gallery,234,131,365,0.641096,0.358904
3,Queensland Family and Child Commission,56,17,73,0.767123,0.232877
4,Queensland Human Rights Commission,35,9,44,0.795455,0.204545


In [21]:
from great_expectations.checkpoint.checkpoint import SimpleCheckpoint

# Review and save our Expectation Suite
print(validator.get_expectation_suite(discard_failed_expectations=True))

validator.save_expectation_suite(discard_failed_expectations=True)

# Set up and run a Simple Checkpoint for ad hoc validation of our data
checkpoint_config = {
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name,
        }
    ],
}

2023-05-23 21:11:35,546 - great_expectations.validator.validator - INFO - 	2 expectation(s) included in expectation_suite.
{
  "expectations": [
    {
      "kwargs": {
        "column": "Other entities"
      },
      "meta": {},
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "kwargs": {
        "column": "other_entities"
      },
      "meta": {},
      "expectation_type": "expect_column_values_to_not_be_null"
    }
  ],
  "meta": {
    "great_expectations_version": "0.16.13"
  },
  "ge_cloud_id": null,
  "expectation_suite_name": "processed_data_suite",
  "data_asset_type": null
}
2023-05-23 21:11:35,549 - great_expectations.validator.validator - INFO - 	2 expectation(s) included in expectation_suite.


In [23]:
checkpoint_name = "processed_data_checkpoint"
checkpoint = gx.checkpoint.SimpleCheckpoint(
    name = checkpoint_name,
    data_context = context,
    **checkpoint_config,
)
checkpoint_result = checkpoint.run()

# Build Data Docs
context.build_data_docs()

2023-05-23 21:12:08,585 - great_expectations.datasource.fluent.fluent_base_model - INFO - PandasS3Datasource.dict() - substituting config values
2023-05-23 21:12:08,587 - great_expectations.datasource.data_connector.batch_filter - INFO - batch_slice: None was parsed to: slice(0, None, None)
2023-05-23 21:12:08,588 - great_expectations.datasource.data_connector.batch_filter - INFO - batch_slice: slice(0, None, None) was parsed to: slice(0, None, None)
2023-05-23 21:12:08,589 - great_expectations.datasource.fluent.fluent_base_model - INFO - CSVAsset.dict() - substituting config values
2023-05-23 21:12:08,926 - great_expectations.validator.validator - INFO - 	1 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{'local_site': 'file://C:\\Users\\gdbt0\\Projects\\Data-preparation-pipeline\\data-processing-pipeline\\great_expectations\\uncommitted/data_docs/local_site/index.html'}

In [24]:
# add (save) the checkpoint to the data context
context.add_checkpoint(checkpoint=checkpoint)
cp = context.get_checkpoint(name=checkpoint_name)
assert cp.name == checkpoint_name

2023-05-23 21:12:11,103 - py.warnings - WARNING - C:\Users\gdbt0\anaconda3\envs\Data-preparation-pipeline\lib\site-packages\great_expectations\data_context\data_context\abstract_data_context.py:1895: DeprecationWarning: A Checkpoint named processed_data_checkpoint already exists.; using add_checkpoint to overwrite an existing value is deprecated as of v0.15.50 and will be removed in v0.18. Please use add_or_update_checkpoint instead.
  warnings.warn(



In [25]:
# Get the only validation_result_identifier from our SimpleCheckpoint run, and open Data Docs to that page
validation_result_identifier = checkpoint_result.list_validation_result_identifiers()[0]
context.open_data_docs(resource_identifier=validation_result_identifier)

In [ ]:
suite.show_expectations_by_expectation_type()

In [ ]:
import sys
# context = gx.get_context()
result = context.run_checkpoint(
    checkpoint_name=checkpoint_name,
)

if not result["success"]:
    print("Validation failed!")
    sys.exit(1)

print("Validation succeeded!")

In [ ]:
validator.expect_column_values_to_not_be_null("Other entities")
validator.expect_column_values_to_be_between("Percentage_Female", auto=True)

In [ ]:
checkpoint = gx.checkpoint.SimpleCheckpoint(
    name="my_quickstart_checkpoint",
    data_context=context,
    validator=validator,
)


In [ ]:
checkpoint_result = checkpoint.run()


In [ ]:
validation_result_identifier = checkpoint_result.list_validation_result_identifiers()[0]
context.open_data_docs(resource_identifier=validation_result_identifier)


In [ ]:
context.build_data_docs()
context.open_data_docs()

